In [3479]:
import numpy as np
from numpy import random
import math

In [3480]:
customer_coords = np.loadtxt('customers_40.data')
swarm_data = {}

In [3481]:
def swarm_init(vars_count, swarm_pop):
    swarm_idx = []
    for i in range(swarm_pop):        
        swarm_idx.append(random.permutation(vars_count))

    return swarm_idx  

In [3482]:
def calculate_route_cost(swarm_idx, coords):
    distances = []
    for particle in swarm_idx:
        total_dist = 0
        for var in range(len(particle) - 1):
            dist = math.sqrt((coords[particle[var]][0] - coords[particle[var + 1]][0])**2 +\
                      (coords[particle[var]][1] - coords[particle[var + 1]][1])**2)
            total_dist += dist
        distances.append(round(total_dist, 3))

    return distances

In [3483]:
def init_swarm_data():
    global swarm_data
    swarm_data['global_best'] = {'particle': None, 'cost': np.inf}
    swarm_data['particle_data'] = []

    for i in range(len(swarm_idx)):
        swarm_data['particle_data'].append({'current': {'particle': None,
                                                        'cost': 0},
                                            'velocity': {'old': {
                                                'seq_start': 0, # starts with local by default
                                                'local_swap': [],
                                                'global_swap': []
                                                },
                                                'new': {
                                                'seq_start': 0, # starts with local by default
                                                'local_swap': [],
                                                'global_swap': []
                                                }},
                                            'best': {'particle': None,
                                                     'cost': np.inf}})
    return swarm_data

In [3484]:
def update_swarm_data(swarm_data, swarm_idx, costs):
    for i in range(len(swarm_idx)):
        swarm_data['particle_data'][i]['current']['particle'] = swarm_idx[i]
        swarm_data['particle_data'][i]['current']['cost'] = costs[i]
        #print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
        #print("Before change: current is "+str(swarm_data['particle_data'][i]['current']['particle'])+" with cost "+str(swarm_data['particle_data'][i]['current']['cost']))
        #print("Before change: local best is "+str(swarm_data['particle_data'][i]['best']['particle'])+" with cost "+str(swarm_data['particle_data'][i]['best']['cost']))
        if swarm_data['particle_data'][i]['current']['cost'] <= swarm_data['particle_data'][i]['best']['cost']:
            #print("New local best at index "+str(i))
            swarm_data['particle_data'][i]['best']['particle'] = swarm_data['particle_data'][i]['current']['particle']
            swarm_data['particle_data'][i]['best']['cost'] = swarm_data['particle_data'][i]['current']['cost']
        if swarm_data['particle_data'][i]['current']['cost'] <= swarm_data['global_best']['cost']:
            #print("New global best at index "+str(i))
            swarm_data['global_best']['particle'] = swarm_data['particle_data'][i]['current']['particle']
            swarm_data['global_best']['cost'] = swarm_data['particle_data'][i]['current']['cost']
        #print("After change: current is "+str(swarm_data['particle_data'][i]['current']['particle'])+" with cost "+str(swarm_data['particle_data'][i]['current']['cost']))
        #print("After change: local best is "+str(swarm_data['particle_data'][i]['best']['particle'])+" with cost "+str(swarm_data['particle_data'][i]['best']['cost']))
        #print("^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^")
    generate_swap_sequence(swarm_data)
    
    return swarm_data

In [3485]:
def generate_swap_sequence(swarm_data):
    global_best = swarm_data['global_best']['particle']
    #print(global_best)
    for particle in swarm_data['particle_data']:
        alpha = random.uniform(0, 1) # local weight
        beta = random.uniform(0, 1) # gloabl weight
        current = particle['current']['particle']
        local_best = particle['best']['particle']
   
        global_swap_seq = []
        local_swap_seq = []
        # Create a working copy of the particle to keep track of local swap updates
        particle_copy = list(current.copy())
        for idx, i in enumerate(particle_copy):
            if i != local_best[idx]:
                target_idx = particle_copy.index(local_best[idx])
                if (target_idx, idx) not in local_swap_seq:
                    local_swap_seq.append((idx, target_idx))
                    # perform temporary swap to keep track of changes as you generate swap sequences
                    temp = i
                    particle_copy[idx] = particle_copy[target_idx]
                    particle_copy[target_idx] = temp
        # Create another working copy of the particle to keep track of global swap updates
        # this is to avoid having the global swap generation being done on an altered copy
        particle_copy = list(current.copy())
        for idx, i in enumerate(particle_copy):
            if i != global_best[idx]:
                target_idx = particle_copy.index(global_best[idx])
                if (target_idx, idx) not in global_swap_seq:
                    global_swap_seq.append((idx, target_idx))
                    # perform temporary swap to keep track of changes as you generate swap sequences
                    temp = i
                    particle_copy[idx] = particle_copy[target_idx]
                    particle_copy[target_idx] = temp

        # set current new velocity to old velocity before setting fresh new velocity
        particle['velocity']['old']['seq_start'] = particle['velocity']['new']['seq_start']
        particle['velocity']['old']['local_swap'] = particle['velocity']['new']['local_swap']
        particle['velocity']['old']['global_swap'] = particle['velocity']['new']['global_swap']
        # 0 represents local and 1 represents global, we randomly choose where overall swap sequence begins
        seq_start = round(random.uniform(0, 1))
        if seq_start == 1 or len(local_swap_seq) == 0:
            particle['velocity']['new']['seq_start'] = 1
        elif seq_start == 0:
            particle['velocity']['new']['seq_start'] = 0
        particle['velocity']['new']['local_swap'] = local_swap_seq[:math.ceil(alpha * len(local_swap_seq))]
        particle['velocity']['new']['global_swap'] = global_swap_seq[:math.ceil(beta * len(global_swap_seq))]
                               
    return

In [3486]:
def run_swap_sequence(swarm_data):
    new_swarm_idx = []

    for particle_data in swarm_data['particle_data']:
        particle = particle_data['current']['particle'].copy()
        old_velocity = particle_data['velocity']['old']
        new_velocity = particle_data['velocity']['new']
        #print("**************************************")
        #print("Particle now:"+ str(particle))
        #print("global swap seq:"+str(new_velocity['global_swap']))
        if old_velocity['seq_start'] == 0:
            for swap in old_velocity['local_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
            # fuction to run swap seq based on new particle
            for swap in old_velocity['global_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
        else:
            for swap in old_velocity['global_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
            for swap in old_velocity['local_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
        if new_velocity['seq_start'] == 0:
            for swap in new_velocity['local_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
            for swap in new_velocity['global_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
        else:
            for swap in new_velocity['global_swap']:     
                #print("Swapping with global "+str(particle[swap[0]])+" with "+str(particle[swap[1]]))
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
                #print("Particle after this swap:"+ str(particle))
            for swap in new_velocity['local_swap']:
                temp = particle[swap[0]]
                particle[swap[0]] = particle[swap[1]]
                particle[swap[1]] = temp
        #print("Particle after all swaps"+str(particle))
        #print("**************************************")
        new_swarm_idx.append(particle)

    return new_swarm_idx

In [3487]:
def travelling_salesman_pso(customer_coords):
    loops = 100
    customer_coords = np.loadtxt('customers_40.data')
    swarm_idx = swarm_init(len(customer_coords), 20)
    swarm_data = init_swarm_data()
    for i in range(loops):
        print("***********************loop "+str(i)+"****************************")
        
        #print("swarm after calculate_route_cost()")
        #print(swarm_data)
        swarm_data = update_swarm_data(swarm_data, swarm_idx, costs)
        print("Costs:"+str(costs))
        print("Best Cost:"+str(swarm_data['global_best']['cost']))
        print("Best Particle:"+str(swarm_data['global_best']['particle']))
        #print("swarm after update_swarm_data()")
        #print(swarm_data)
        
        swarm_idx = run_swap_sequence(swarm_data)
        #print("swarm after run_swap_sequence()")
        #print(swarm_data)
        print("****************************************************")
    return

In [3488]:
travelling_salesman_pso(len(customer_coords))

***********************loop 0****************************
Costs:[2915.966, 3167.161, 3106.513, 3066.517, 3433.137, 2988.332, 2938.614, 2868.743, 3128.325, 2929.753, 3270.63, 3298.731, 2918.06, 3453.893, 3266.715, 2685.514, 3167.633, 3034.544, 2943.243, 2852.429]
Best Cost:2685.514
Best Particle:[33 19 14 58 43 22 12  2 29 26 36 53 49  8  6  1 59 38 44 24 56 11 27  3
 52 30 47 40 32 21 20 39 48 55  0 51  4 15 16 45  9  7 42  5 57 37 50 46
 54 25 10 23 35 41 18 31 28 34 13 17]
****************************************************
***********************loop 1****************************
Costs:[2781.837, 2608.9, 2768.255, 2982.969, 2792.834, 2704.825, 2612.586, 3022.023, 2671.114, 2637.136, 2797.831, 2685.514, 3022.41, 2649.775, 2865.934, 2685.514, 3023.793, 2821.807, 2782.816, 2789.446]
Best Cost:2608.9
Best Particle:[33 19 14 58 43 22 12  2 29 26 36 53 49  8  6  1 59 38 44 24 56 11 27  3
 52 47 17 20 32 39 30 41 35 31 46  7 45 40 54 16  5 37  9 18 25 48 10 23
 28 15  4 42 13 55  0 21 57 